## SparkSession Build

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 148kB/s eta 0:00:01   |█▏                              | 8.0MB 2.6MB/s eta 0:01:20     |███                             | 20.5MB 1.0MB/s eta 0:03:07     |███▍                            | 22.5MB 1.0MB/s eta 0:03:05     |███████▏                        | 47.5MB 2.6MB/s eta 0:01:03     |██████████                      | 66.2MB 5.4MB/s eta 0:00:28     |███████████▍                    | 75.2MB 684kB/s eta 0:03:21     |███████████▍                    | 75.4MB 684kB/s eta 0:03:21     |███████████▊                    | 78.0MB 684kB/s eta 0:03:17     |███████████████                 | 99.6MB 4.8MB/s eta 0:00:24     |███████████████▏                | 100.4MB 4.8MB/s eta 0:00:24     |█████████████████               | 112.5MB 5.3MB/s eta 0:00:19     |█████████████████               | 112.6MB 5.3MB/s eta 0:00:19     |█████████████████▎              | 114.9MB 4.9MB/s eta 0:00:21     |████████████████████▍           | 135.6MB 3.0MB/s eta 0:00:26     |█

In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("BoazTestApp").master("spark://master:7077").config("spark.cores.max", "4").config("spark.sql.shuffle.partitions", "5").getOrCreate()
spark

In [ ]:
# spark on hadoop yarn
spark = SparkSession.builder.appName("BoazTestApp").master("yarn").config("spark.cores.max", "4").config("spark.sql.shuffle.partitions", "5").getOrCreate()
spark

## Read Data

In [3]:
staticDataFrame = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('./data/retail-data/by-day/*.csv')
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

AnalysisException: Path does not exist: hdfs://master:9000/user/root/data/retail-data/by-day/*.csv;

In [6]:
staticSchema

StructType(List(StructField(InvoiceNo,StringType,true),StructField(StockCode,StringType,true),StructField(Description,StringType,true),StructField(Quantity,IntegerType,true),StructField(InvoiceDate,StringType,true),StructField(UnitPrice,DoubleType,true),StructField(CustomerID,DoubleType,true),StructField(Country,StringType,true)))

In [7]:
staticDataFrame.show(5, False)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+
|580538   |23084    |RABBIT NIGHT LIGHT             |48      |2011-12-05 08:38:00|1.79     |14075.0   |United Kingdom|
|580538   |23077    |DOUGHNUT LIP GLOSS             |20      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22906    |12 MESSAGE CARDS WITH ENVELOPES|24      |2011-12-05 08:38:00|1.65     |14075.0   |United Kingdom|
|580538   |21914    |BLUE HARMONICA IN BOX          |24      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|
|580538   |22467    |GUMBALL COAT RACK              |6       |2011-12-05 08:38:00|2.55     |14075.0   |United Kingdom|
+---------+---------+---------------------------

## Preprocessing

In [8]:
from pyspark.sql.functions import date_format, col

#### Fill NA

In [9]:
preppedDataFrame = staticDataFrame.na.fill(0).withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE")).coalesce(5)

In [10]:
preppedDataFrame.show(5, False)

+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|Description                    |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |day_of_week|
+---------+---------+-------------------------------+--------+-------------------+---------+----------+--------------+-----------+
|580538   |23084    |RABBIT NIGHT LIGHT             |48      |2011-12-05 08:38:00|1.79     |14075.0   |United Kingdom|Monday     |
|580538   |23077    |DOUGHNUT LIP GLOSS             |20      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|Monday     |
|580538   |22906    |12 MESSAGE CARDS WITH ENVELOPES|24      |2011-12-05 08:38:00|1.65     |14075.0   |United Kingdom|Monday     |
|580538   |21914    |BLUE HARMONICA IN BOX          |24      |2011-12-05 08:38:00|1.25     |14075.0   |United Kingdom|Monday     |
|580538   |22467    |GUMBALL COAT RACK              |6       |2011-12-05 08:38:00|2

#### Split Data

In [11]:
trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

#### Data Pipeline

In [12]:
!pip install numpy

     |████████████████████████████████| 15.4MB 287kB/s eta 0:00:01    |███▌                            | 1.7MB 1.1MB/s eta 0:00:13     |██████████████████████▌         | 10.8MB 691kB/s eta 0:00:07     |████████████████████████████▎   | 13.6MB 414kB/s eta 0:00:05


In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [14]:
indexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")
encoder = OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")
vectorAssembler = VectorAssembler().setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]).setOutputCol("features")

In [15]:
transformationPipeline = Pipeline().setStages([indexer, encoder, vectorAssembler])

In [16]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

#### Transformed Data with pipeline

In [17]:
transformedTrain = fittedPipeline.transform(trainDataFrame)
transformedTest = fittedPipeline.transform(testDataFrame)

In [18]:
transformedTrain.show(5, False)

+---------+---------+------------------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+---------------------------+
|InvoiceNo|StockCode|Description                   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |day_of_week|day_of_week_index|day_of_week_encoded|features                   |
+---------+---------+------------------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+---------------------------+
|537226   |22811    |SET OF 6 T-LIGHTS CACTI       |6       |2010-12-06 08:34:00|2.95     |15987.0   |United Kingdom|Monday     |2.0              |(5,[2],[1.0])      |(7,[0,1,4],[2.95,6.0,1.0]) |
|537226   |21713    |CITRONELLA CANDLE FLOWERPOT   |8       |2010-12-06 08:34:00|2.1      |15987.0   |United Kingdom|Monday     |2.0              |(5,[2],[1.0])      |(7,[0,1,4],[2.1,8.0,1.0])  |
|537226   |22927    

## Model Build

In [17]:
from pyspark.ml.clustering import KMeans

In [18]:
kmeans = KMeans().setK(20).setSeed(1)
kmModel = kmeans.fit(transformedTrain)

In [19]:
kmModel.summary.trainingCost

75301118.84698008

## Prediction

In [20]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [21]:
predictions = kmModel.transform(transformedTest)

In [22]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.5427938390491533


In [23]:
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[4.09293606 2.73959977 0.18896861 0.19629835 0.18589279 0.16698473
 0.14731972]
[1.0400e+00 7.4215e+04 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00]
[ 1.0400e+00 -7.4215e+04  0.0000e+00  1.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00]
[ 3.897e+04 -1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
  1.000e+00]
[ 1.6670865e+04 -1.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  1.0000000e+00  0.0000000e+00]
[ 7.5000e-03 -9.4045e+03  2.5000e-01  7.5000e-01  0.0000e+00  0.0000e+00
  0.0000e+00]
[ 7.385808e+03 -6.000000e-01  0.000000e+00  8.000000e-01  2.000000e-01
  0.000000e+00  0.000000e+00]
[ 1.94092118e+03 -1.76470588e-01  5.88235294e-02  1.76470588e-01
  4.11764706e-01  0.00000000e+00  3.52941176e-01]
[8.407500e-01 1.213475e+03 2.125000e-01 2.500000e-01 1.125000e-01
 2.000000e-01 1.625000e-01]
[ 2.91481481e-01 -1.34285185e+03  1.85185185e-01  2.96296296e-01
  1.85185185e-01  2.59259259e-01  7.40740741e-02]
[  1.41599732 114.6946616    0.2081